In [82]:
## Question 1
from math import sqrt

# Data for each product: Demand (D), Common Ordering Cost (C), Specific Ordering Cost (S), Unit Cost (P), Holding Cost Rate (H)
products = [
    {'D': 1000, 'C': 100, 'S': 10, 'P': 50, 'H': 0.2},
    {'D': 300, 'C': 100, 'S': 20, 'P': 60, 'H': 0.2},
    {'D': 100, 'C': 100, 'S': 25, 'P': 30, 'H': 0.2},
    {'D': 50, 'C': 100, 'S': 25, 'P': 30, 'H': 0.2}
]

# Independent Sourcing Strategy
def independent_sourcing_cost(products):
    total_cost = 0
    for product in products:
        S = product['C'] + product['S']  # Total Ordering Cost
        H = product['H'] * product['P']  # Holding Cost per unit per year
        Q = sqrt((2 * product['D'] * S) / H)  # EOQ
        ordering_cost = product['D'] * S / Q 
        holding_cost = H * Q / 2
        total_cost += ordering_cost + holding_cost
    return total_cost

# Calculate each cost
independent_cost = independent_sourcing_cost(products)
independent_cost

3073.9153138822376

In [83]:
## Question 2

def joint_sourcing_cost(products):
    # Assuming C is the common fixed cost and should only be counted once
    C_common = products[0]['C']  # Assuming all products have the same common fixed cost
    S_total = C_common + sum(product['S'] for product in products)  # Fixed + variable costs
    
    # Calculate the inner part of the order frequency formula
    inner = sum(product['D'] * product['H'] * product['P'] for product in products) / (2 * S_total)
    
    # Calculate the joint order frequency
    n = sqrt(inner)
    
    # Calculate total ordering cost based on the joint order frequency
    total_ordering_cost = n * S_total
    
    # Calculate total holding cost for all products
    total_holding_cost = 0
    for product in products:
        Q = product['D'] / n  # Order quantity for each product
        holding_cost = Q * product['H'] * product['P'] / 2  # Holding cost for each product
        total_holding_cost += holding_cost

    # Return the sum of total ordering and holding costs
    return total_ordering_cost + total_holding_cost
joint_cost = joint_sourcing_cost(products)
joint_cost

2284.7319317591728

In [115]:
## Question 3
import pandas as pd
products = pd.DataFrame(products)

# Most Frequent Product 
freq_product = products.iloc[0] # based on Q1
S_freq = freq_product['C'] + freq_product['S'] 
H_freq = freq_product['H'] * freq_product['P']
EOQ_freq = sqrt((2 * freq_product['D'] * S_freq) / H_freq)
freq_freq = freq_product['D'] / EOQ_freq 
products.loc[0, 'freq'] = freq_freq
products.loc[0, 'm_i'] = 1

products_ex = products.iloc[1:]
products_ex['Q'] = (2 * products_ex['D'] * products_ex['S'] / (products_ex['H'] * products_ex['P'])) ** 0.5
products_ex['freq'] = (products_ex['D']/products_ex['Q'])
products_ex['m_i'] = round(freq_freq / products_ex['freq'], 0)
products.iloc[1:] = products_ex
products

C:\Users\Younkyung Lee\AppData\Local\Temp\ipykernel_29192\2701420217.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_ex['Q'] = (2 * products_ex['D'] * products_ex['S'] / (products_ex['H'] * products_ex['P'])) ** 0.5
C:\Users\Younkyung Lee\AppData\Local\Temp\ipykernel_29192\2701420217.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_ex['freq'] = (products_ex['D']/products_ex['Q'])
C:\Users\Younkyung Lee\AppData\Local\Temp\ipykernel_29192\2701420217.py:17: SettingWithCopyWarning: 
A

,D,C,S,P,H,freq,m_i
0,1000,100,10,50,0.2,6.741999,1.0
1,300,100,20,60,0.2,9.486833,1.0
2,100,100,25,30,0.2,3.464102,2.0
3,50,100,25,30,0.2,2.449490,3.0


In [116]:
# Recalculate the ordering frequency 'n' of the most frequently ordered product
h = products['H']
C = products['P']
D = products['D']
m = products['m_i']
S = 100
s = products['S']

n_updated = ((h * C * D * m).sum() / (2 * (S + (s/m).sum())))**(1/2)

# new frequency
products.loc[0, 'nfreq'] = n_updated
products.loc[1:3, 'nfreq'] = n_updated / products['m_i']

# new Q
products['nQ'] = products['D']/products['nfreq']

# Calculate the total cost
holding_cost = (products['nQ']*products['H']*products['P']/2).sum()
ordering_cost = (products['D'] *products['S']/products['nQ']).sum()

holding_cost + ordering_cost + products['nfreq'].max()*100

2176.2735734890193